In [3]:
import src.dataset as ds

In [4]:
x,y,tokenizer,songs = ds.load_tokenized_data(with_melody=True, melody_type='kaki')

Loading the songs embedding:  15%|██████████████▌                                                                                   | 91/614 [00:15<01:43,  5.07it/s]

Couldn't load .\Data\midi_files\david_bowie_-_lazarus.mid, issue with the midi file.


Loading the songs embedding:  19%|██████████████████▏                                                                              | 115/614 [00:19<01:28,  5.66it/s]

Couldn't load .\Data\midi_files\beastie_boys_-_girls.mid, issue with the midi file.


Loading the songs embedding:  20%|██████████████████▉                                                                              | 120/614 [00:20<01:16,  6.50it/s]

...

Couldn't load .\Data\midi_files\brian_mcknight_-_on_the_down_low.mid, issue with the midi file.


Loading the songs embedding:  93%|██████████████████████████████████████████████████████████████████████████████████████████▌      | 573/614 [01:32<00:06,  6.14it/s]

Couldn't load .\Data\midi_files\aaron_neville_-_tell_it_like_it_is.mid, issue with the midi file.


Loading the songs embedding: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [01:38<00:00,  5.91it/s]


MemoryError: 